In [37]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [81]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 128 # 1st layer num features
num_hidden_2 = 32 # 2nd layer num features (the latent dim)
num_input = 86 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [89]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [99]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VHc.csv', delimiter=',')
my_data1 = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VVc.csv', delimiter=',')

In [100]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
print(data.shape)
print(data.shape)

(291, 43)
(291, 43)


In [92]:
print(data.max())

0.162946105003357


In [93]:
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(291, 43)
(291, 43)


In [94]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [95]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [96]:
p = np.array(ctype)
print(p.shape)

(292,)


In [97]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [98]:
label = np.array(label)
print(label.shape)

(291,)


In [101]:
data2 = np.concatenate((data,data1), axis=1)
print(data2.shape)

(291, 86)


In [102]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:232,:]
X_test = data2[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 86)
(59, 86)
(232,)
(59,)


In [103]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [104]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.55529028
Epoch: 2 cost = 0.53730622
Epoch: 3 cost = 0.52514109
Epoch: 4 cost = 0.51607392
Epoch: 5 cost = 0.50841832
Epoch: 6 cost = 0.50124759
Epoch: 7 cost = 0.49410008
Epoch: 8 cost = 0.48676513
Epoch: 9 cost = 0.47915759
Epoch: 10 cost = 0.47125340
Epoch: 11 cost = 0.46306317
Epoch: 12 cost = 0.45462370
Epoch: 13 cost = 0.44599865
Epoch: 14 cost = 0.43728181
Epoch: 15 cost = 0.42859864
Epoch: 16 cost = 0.42010203
Epoch: 17 cost = 0.41196064
Epoch: 18 cost = 0.40434216
Epoch: 19 cost = 0.39739495
Epoch: 20 cost = 0.39123299
Epoch: 21 cost = 0.38592447
Epoch: 22 cost = 0.38148544
Epoch: 23 cost = 0.37787524
Epoch: 24 cost = 0.37499532
Epoch: 25 cost = 0.37269211
Epoch: 26 cost = 0.37077242
Epoch: 27 cost = 0.36903864
Epoch: 28 cost = 0.36732815
Epoch: 29 cost = 0.36553045
Epoch: 30 cost = 0.36357683
Epoch: 31 cost = 0.36142133
Epoch: 32 cost = 0.35902518
Epoch: 33 cost = 0.35635312
Epoch: 34 cost = 0.35337531
Epoch: 35 cost = 0.35008467
Epoch: 36 cost = 0.34652332
E

Epoch: 326 cost = 0.00381074
Epoch: 327 cost = 0.00372341
Epoch: 328 cost = 0.00374917
Epoch: 329 cost = 0.00367784
Epoch: 330 cost = 0.00367770
Epoch: 331 cost = 0.00374779
Epoch: 332 cost = 0.00374401
Epoch: 333 cost = 0.00453202
Epoch: 334 cost = 0.12063776
Epoch: 335 cost = 0.00563411
Epoch: 336 cost = 0.00420728
Epoch: 337 cost = 0.00398582
Epoch: 338 cost = 0.00383294
Epoch: 339 cost = 0.00374335
Epoch: 340 cost = 0.00370830
Epoch: 341 cost = 0.00366839
Epoch: 342 cost = 0.00360567
Epoch: 343 cost = 0.00360213
Epoch: 344 cost = 0.00356685
Epoch: 345 cost = 0.00354588
Epoch: 346 cost = 0.00358432
Epoch: 347 cost = 0.00357701
Epoch: 348 cost = 0.00353742
Epoch: 349 cost = 0.00347282
Epoch: 350 cost = 0.00344683
Epoch: 351 cost = 0.00346959
Epoch: 352 cost = 0.00343962
Epoch: 353 cost = 0.00345804
Epoch: 354 cost = 0.00341580
Epoch: 355 cost = 0.00343455
Epoch: 356 cost = 0.00344338
Epoch: 357 cost = 0.00339145
Epoch: 358 cost = 0.00342506
Epoch: 359 cost = 0.00343116
Epoch: 360 cos

Epoch: 636 cost = 0.00168499
Epoch: 637 cost = 0.00169512
Epoch: 638 cost = 0.00168368
Epoch: 639 cost = 0.00167578
Epoch: 640 cost = 0.00167059
Epoch: 641 cost = 0.00167943
Epoch: 642 cost = 0.00167783
Epoch: 643 cost = 0.00168410
Epoch: 644 cost = 0.00168808
Epoch: 645 cost = 0.00170505
Epoch: 646 cost = 0.00172357
Epoch: 647 cost = 0.00167363
Epoch: 648 cost = 0.00171233
Epoch: 649 cost = 0.00169626
Epoch: 650 cost = 0.00170451
Epoch: 651 cost = 0.00166849
Epoch: 652 cost = 0.00169745
Epoch: 653 cost = 0.00172645
Epoch: 654 cost = 0.00167704
Epoch: 655 cost = 0.00169076
Epoch: 656 cost = 0.00169212
Epoch: 657 cost = 0.00164823
Epoch: 658 cost = 0.00169265
Epoch: 659 cost = 0.00170269
Epoch: 660 cost = 0.00169964
Epoch: 661 cost = 0.00173418
Epoch: 662 cost = 0.00166920
Epoch: 663 cost = 0.00166007
Epoch: 664 cost = 0.00166961
Epoch: 665 cost = 0.00168605
Epoch: 666 cost = 0.00166534
Epoch: 667 cost = 0.00169735
Epoch: 668 cost = 0.00171182
Epoch: 669 cost = 0.00170467
Epoch: 670 cos

Epoch: 952 cost = 0.00165619
Epoch: 953 cost = 0.00164317
Epoch: 954 cost = 0.00164916
Epoch: 955 cost = 0.00165156
Epoch: 956 cost = 0.00170185
Epoch: 957 cost = 0.00165426
Epoch: 958 cost = 0.00164290
Epoch: 959 cost = 0.00163378
Epoch: 960 cost = 0.00164477
Epoch: 961 cost = 0.00167189
Epoch: 962 cost = 0.00166239
Epoch: 963 cost = 0.00163614
Epoch: 964 cost = 0.00166184
Epoch: 965 cost = 0.00167993
Epoch: 966 cost = 0.00166595
Epoch: 967 cost = 0.00164048
Epoch: 968 cost = 0.00164759
Epoch: 969 cost = 0.00163050
Epoch: 970 cost = 0.00163420
Epoch: 971 cost = 0.00166202
Epoch: 972 cost = 0.00167956
Epoch: 973 cost = 0.00166856
Epoch: 974 cost = 0.00165842
Epoch: 975 cost = 0.00164331
Epoch: 976 cost = 0.00166113
Epoch: 977 cost = 0.00162986
Epoch: 978 cost = 0.00164650
Epoch: 979 cost = 0.00165813
Epoch: 980 cost = 0.00168793
Epoch: 981 cost = 0.00166069
Epoch: 982 cost = 0.00171036
Epoch: 983 cost = 0.00164392
Epoch: 984 cost = 0.00166835
Epoch: 985 cost = 0.00168304
Epoch: 986 cos

In [105]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [106]:
print(y_test.shape)

(59, 5)


In [107]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.66101694


In [64]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:204,:]
X_test = data2[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 86)
(87, 86)
(204,)
(87,)


In [65]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [77]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.22052824
Epoch: 2 cost = 1.11546882
Epoch: 3 cost = 1.04317788
Epoch: 4 cost = 0.98644296
Epoch: 5 cost = 0.93291008
Epoch: 6 cost = 0.87786917
Epoch: 7 cost = 0.82054577
Epoch: 8 cost = 0.76160373
Epoch: 9 cost = 0.70224400
Epoch: 10 cost = 0.64409655
Epoch: 11 cost = 0.58960698
Epoch: 12 cost = 0.54180586
Epoch: 13 cost = 0.50284666
Epoch: 14 cost = 0.47353572
Epoch: 15 cost = 0.45380084
Epoch: 16 cost = 0.44204942
Epoch: 17 cost = 0.43537249
Epoch: 18 cost = 0.43114958
Epoch: 19 cost = 0.42784766
Epoch: 20 cost = 0.42474677
Epoch: 21 cost = 0.42154036
Epoch: 22 cost = 0.41814295
Epoch: 23 cost = 0.41462596
Epoch: 24 cost = 0.41118956
Epoch: 25 cost = 0.40810863
Epoch: 26 cost = 0.40560766
Epoch: 27 cost = 0.40371831
Epoch: 28 cost = 0.40227097
Epoch: 29 cost = 0.40101673
Epoch: 30 cost = 0.39973148
Epoch: 31 cost = 0.39826614
Epoch: 32 cost = 0.39655395
Epoch: 33 cost = 0.39458674
Epoch: 34 cost = 0.39238912
Epoch: 35 cost = 0.39000010
Epoch: 36 cost = 0.38746023
E

Epoch: 339 cost = 0.00401818
Epoch: 340 cost = 0.00395990
Epoch: 341 cost = 0.00394195
Epoch: 342 cost = 0.00398119
Epoch: 343 cost = 0.00390667
Epoch: 344 cost = 0.00394201
Epoch: 345 cost = 0.00389476
Epoch: 346 cost = 0.00393639
Epoch: 347 cost = 0.00396090
Epoch: 348 cost = 0.00386870
Epoch: 349 cost = 0.00384974
Epoch: 350 cost = 0.00389905
Epoch: 351 cost = 0.00393387
Epoch: 352 cost = 0.00398307
Epoch: 353 cost = 0.00391697
Epoch: 354 cost = 0.00388925
Epoch: 355 cost = 0.00388346
Epoch: 356 cost = 0.00385883
Epoch: 357 cost = 0.00385351
Epoch: 358 cost = 0.00385472
Epoch: 359 cost = 0.00382210
Epoch: 360 cost = 0.00383419
Epoch: 361 cost = 0.00383880
Epoch: 362 cost = 0.00381514
Epoch: 363 cost = 0.00392102
Epoch: 364 cost = 0.00390894
Epoch: 365 cost = 0.00384995
Epoch: 366 cost = 0.00385292
Epoch: 367 cost = 0.00384735
Epoch: 368 cost = 0.00389395
Epoch: 369 cost = 0.00382203
Epoch: 370 cost = 0.00385506
Epoch: 371 cost = 0.00387748
Epoch: 372 cost = 0.00394622
Epoch: 373 cos

Epoch: 662 cost = 0.00369214
Epoch: 663 cost = 0.00369661
Epoch: 664 cost = 0.00367305
Epoch: 665 cost = 0.00367118
Epoch: 666 cost = 0.00364348
Epoch: 667 cost = 0.00363887
Epoch: 668 cost = 0.00366672
Epoch: 669 cost = 0.00368747
Epoch: 670 cost = 0.00374039
Epoch: 671 cost = 0.00365766
Epoch: 672 cost = 0.00364373
Epoch: 673 cost = 0.00363334
Epoch: 674 cost = 0.00373366
Epoch: 675 cost = 0.00366732
Epoch: 676 cost = 0.00365330
Epoch: 677 cost = 0.00362456
Epoch: 678 cost = 0.00363823
Epoch: 679 cost = 0.00371921
Epoch: 680 cost = 0.00365081
Epoch: 681 cost = 0.00365280
Epoch: 682 cost = 0.00362575
Epoch: 683 cost = 0.00363331
Epoch: 684 cost = 0.00371535
Epoch: 685 cost = 0.00378639
Epoch: 686 cost = 0.00358587
Epoch: 687 cost = 0.00363126
Epoch: 688 cost = 0.00363469
Epoch: 689 cost = 0.00370567
Epoch: 690 cost = 0.00371645
Epoch: 691 cost = 0.00369377
Epoch: 692 cost = 0.00369196
Epoch: 693 cost = 0.00362015
Epoch: 694 cost = 0.00364433
Epoch: 695 cost = 0.00363302
Epoch: 696 cos

Epoch: 977 cost = 0.00365710
Epoch: 978 cost = 0.00360108
Epoch: 979 cost = 0.00357492
Epoch: 980 cost = 0.00358183
Epoch: 981 cost = 0.00358884
Epoch: 982 cost = 0.00356184
Epoch: 983 cost = 0.00354307
Epoch: 984 cost = 0.00355974
Epoch: 985 cost = 0.00362672
Epoch: 986 cost = 0.00357556
Epoch: 987 cost = 0.00362297
Epoch: 988 cost = 0.00361328
Epoch: 989 cost = 0.00355180
Epoch: 990 cost = 0.00355762
Epoch: 991 cost = 0.00360458
Epoch: 992 cost = 0.00359926
Epoch: 993 cost = 0.00360704
Epoch: 994 cost = 0.00357035
Epoch: 995 cost = 0.00356186
Epoch: 996 cost = 0.00356258
Epoch: 997 cost = 0.00361497
Epoch: 998 cost = 0.00358323
Epoch: 999 cost = 0.00361211
Epoch: 1000 cost = 0.00357538


In [80]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.6551724
